In [14]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns 

In [15]:
# graficos - configuración
sns.set_theme(style="whitegrid")
pd.set_option('display.width', 1000)

try:
    df = pd.read_excel("data/online_retail_II.xlsx")
    print("Archivo cargado exitosamente")
except Exception as e:
    print(f"Error al cargar el archivo: {e}")

print(df.head()) 
print(df.info()) # tipos de variables
print(df.describe().T.round(2)) # estadísticas descriptivas básica

Archivo cargado exitosamente
  Invoice StockCode                          Description  Quantity         InvoiceDate  Price  Customer ID         Country
0  489434     85048  15CM CHRISTMAS GLASS BALL 20 LIGHTS        12 2009-12-01 07:45:00   6.95      13085.0  United Kingdom
1  489434    79323P                   PINK CHERRY LIGHTS        12 2009-12-01 07:45:00   6.75      13085.0  United Kingdom
2  489434    79323W                  WHITE CHERRY LIGHTS        12 2009-12-01 07:45:00   6.75      13085.0  United Kingdom
3  489434     22041         RECORD FRAME 7" SINGLE SIZE         48 2009-12-01 07:45:00   2.10      13085.0  United Kingdom
4  489434     21232       STRAWBERRY CERAMIC TRINKET BOX        24 2009-12-01 07:45:00   1.25      13085.0  United Kingdom
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 525461 entries, 0 to 525460
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   Invoice      52546

In [19]:
df_limpio = df[(df['Quantity'] > 0) & (df['Price'] > 0)].copy()

print(f"Registros originales: {df.shape[0]}")
print(f"Registros limpios: {df_limpio.shape[0]}")
print(f"Registros eliminados: {df.shape[0] - df_limpio.shape[0]}")

Registros originales: 525461
Registros limpios: 511566
Registros eliminados: 13895


In [25]:
# ajuste de tipo de datos, Customer ID e Invoice ahora son tipo String
df_limpio['Customer ID'] = df_limpio['Customer ID'].astype(str)
df_limpio['Invoice'] = df_limpio['Invoice'].astype(str)


In [30]:
print("--- ANÁLISIS DE NULOS ---")
print(df_limpio.isnull().sum())

df_limpio['Customer ID'] = df_limpio['Customer ID'].replace('nan', 'Guest')
df_limpio['Customer ID'] = df_limpio['Customer ID'].fillna('Guest')

df_limpio['Total_Venta'] = df_limpio['Quantity'] * df_limpio['Price']

guest_count = df_limpio[df_limpio['Customer ID'] == 'Guest'].shape[0]
print(f"\nNúmero de transacciones realizadas por invitados (Guest): {guest_count}")


venta_por_tipo = df_limpio.groupby(df_limpio['Customer ID'] == 'Guest')['Total_Venta'].sum()

print("\n--- IMPACTO ECONÓMICO ---")
print(f"Ventas de Usuarios Registrados: ${venta_por_tipo[False]:,.2f}")
print(f"Ventas de Usuarios Invitados (Guest): ${venta_por_tipo[True]:,.2f}")

# Cálculo del porcentaje
porcentaje_guest = (venta_por_tipo[True] / df_limpio['Total_Venta'].sum()) * 100
print(f"Los invitados representan el {porcentaje_guest:.2f}% de los ingresos totales.")

df_limpio[df_limpio['Customer ID'] == 'Guest']

--- ANÁLISIS DE NULOS ---
Invoice        0
StockCode      0
Description    0
Quantity       0
InvoiceDate    0
Price          0
Customer ID    0
Country        0
Total_Venta    0
dtype: int64

Número de transacciones realizadas por invitados (Guest): 103902

--- IMPACTO ECONÓMICO ---
Ventas de Usuarios Registrados: $8,832,003.27
Ventas de Usuarios Invitados (Guest): $1,474,262.32
Los invitados representan el 14.30% de los ingresos totales.


,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,Total_Venta
577,489525,85226C,BLUE PULL BACK RACING CAR,1,2009-12-01 11:49:00,0.55,Guest,United Kingdom,0.55
578,489525,85227,SET/6 3D KIT CARDS FOR KIDS,1,2009-12-01 11:49:00,0.85,Guest,United Kingdom,0.85
1055,489548,22271,FELTCRAFT DOLL ROSIE,1,2009-12-01 12:32:00,2.95,Guest,United Kingdom,2.95
1056,489548,22254,FELT TOADSTOOL LARGE,12,2009-12-01 12:32:00,1.25,Guest,United Kingdom,15.00
1057,489548,22273,FELTCRAFT DOLL MOLLY,3,2009-12-01 12:32:00,2.95,Guest,United Kingdom,8.85
...,...,...,...,...,...,...,...,...,...
525143,538154,82599,FANNY'S REST STOPMETAL SIGN,1,2010-12-09 16:35:00,4.21,Guest,United Kingdom,4.21
525144,538154,84029E,RED WOOLLY HOTTIE WHITE HEART.,5,2010-12-09 16:35:00,8.47,Guest,United Kingdom,42.35
525145,538154,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,4,2010-12-09 16:35:00,8.47,Guest,United Kingdom,33.88
525146,538154,85099B,JUMBO BAG RED RETROSPOT,1,2010-12-09 16:35:00,4.21,Guest,United Kingdom,4.21
